## The Save Westeros Problem

is a subclass of the SearchProblem ADT

In [ ]:
class SwProblem(Problem):

It overides the five instance variables/methods to represent the context of the project


1. set of operators, or actions, available to the agent.

This method returns only operators that are available to the agent in the current state. For example he can only move forward if there's no white walker/obstacle blocking the path and he's not at the boundary of the grid. He can only kill if there are white_walkers surrounding him and he has some dragon glass.

In [ ]:
    # 1. A set of operators, or actions, available to the agent.
    def valid_operators(self, state):
        """list of **valid** action to be performed on state"""
        operators = list()
        x, y = state.location()

        if state.grid.is_safe_location((x - 1, y)):
            operators.append(Operators.LEFT)

        if state.grid.is_safe_location((x + 1, y)):
            operators.append(Operators.RIGHT)

        if state.grid.is_safe_location((x, y - 1)):
            operators.append(Operators.FORWARD)

        if state.grid.is_safe_location((x, y + 1)):
            operators.append(Operators.BACKWARD)

        has_dragon_glass = state.dragon_glass() > 0
        ww_locations = state.grid.nearby_white_walkers(state.location())
        ww_surrounding = len(ww_locations) > 0

        if has_dragon_glass and ww_surrounding:
            operators.append(Operators.KILL)
        return operators

    2. An initial state.
    
  The initial state is initialised by the constructor automatically. Every state has an instance of the grid, since the grid changes according to each operator done on the state

In [ ]:

    # 2. An initial state.
    initial_state = None

    # *. Problem Constructor
    def __init__(self, grid):
        self.initial_state = SwState(
            grid=grid,
        )

    3. A state space: the set of states reachable from the initial state by any sequence of actions.

In lieu of a state space, a `result` method is defined which defines a resultant state from the application of an operator on the current state. If the operator is a moving operator (i.e LEFT, RIGHT, FORWARD, BACKWARD). Then assign this new location to the child state. Else we know that only other operator is KILL, then we kill all the white walkers in the adjacent locations in the subsquent state. We don't need to check these actions since they are only performed if they are valid as the previous method ensures that.

In [ ]:
    # 3. A state space: the set of states reachable from the initial state by
    # any sequence of actions.
    def result(self, state, operator):
        """resultant state from performing the given operator on the given state."""
        moving_operators = [Operators.LEFT, Operators.RIGHT, Operators.FORWARD, Operators.BACKWARD]
        if operator in moving_operators:
            x, y = state.location()

            if operator == Operators.LEFT:
                x -= 1
            if operator == Operators.RIGHT:
                x += 1
            if operator == Operators.FORWARD:
                y -= 1
            if operator == Operators.BACKWARD:
                y += 1

            new_location = x, y
            return SwState(
                grid=state.grid,
                location=new_location,
            )

        else:
            return SwState(
                grid=state.grid,
                white_walkers=True,
            )

    4. A goal test, which the agent applies to a state to determine if it is a goal state.
The function returns true if the state has no white walkers

In [ ]:
    # 4. A goal test, which the agent applies to a state to determine if it is
    # a goal state.
    def goal_test(self, state):
        """goal predicate"""
        return state.white_walkers() == 0

    5. A path cost function: a function that assigns cost to an action
This is done by defining a dictionary, which contains the respective costs to each operator

In [ ]:
    # 5. A path cost function: a function that assigns cost to an action
    def cost_function(self, action):
        """sum of action sequence"""
        cost_dict = dict([
            (Operators.LEFT, 2),
            (Operators.RIGHT, 2),
            (Operators.FORWARD, 2),
            (Operators.BACKWARD, 2),
            (Operators.KILL, 3),
        ])
        return cost_dict[action]